# L4: Tool Calling

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Import libraries

In [2]:
from ai21 import AI21Client
from ai21.logger import set_verbose
from ai21.models.chat import ChatMessage, ToolMessage, FunctionToolDefinition, ToolDefinition, ToolParameters
import json, requests, re
from bs4 import BeautifulSoup

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>utils.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

## Load API key and create AI21Client

In [3]:
from utils import get_ai21_api_key
ai21_api_key = get_ai21_api_key()

client = AI21Client(api_key=ai21_api_key)

# Arithmetic tools for Jamba

In [4]:
def multiplication(a: float, b: float) -> float:
    print(f"Multiplying {a} and {b}...")
    return a * b

def addition(a: float, b: float) -> float:
    print(f"Adding {a} and {b}...")
    return a + b

In [5]:
multiplication_tool = ToolDefinition(
    type="function",
    function=FunctionToolDefinition(
        name="multiplication",
        description="Multiply two numbers",
        parameters=ToolParameters(
            type="object",
            properties={
                "a": {"type": "number", "description": "The first number to multiply"},
                "b": {"type": "number", "description": "The second number to multiply"}
            },
            required=["a", "b"],
        ),
    ),
)

In [6]:
addition_tool = ToolDefinition(
    type="function",
    function=FunctionToolDefinition(
        name="addition",
        description="Add two numbers",
        parameters=ToolParameters(
            type="object",
            properties={
                "a": {"type": "number", "description": "The first number to add"},
                "b": {"type": "number", "description": "The second number to add"}
            },
            required=["a", "b"],
        ),
    ),
)

In [7]:
tools = [multiplication_tool, addition_tool]

In [8]:
messages = [
    ChatMessage(role="system", content="You are a helpful math assistant. Use the supplied tools to assist the user."),
    ChatMessage(role="user", content="What is the capital of France")
]

In [9]:
response = client.chat.completions.create(
    messages=messages, 
    model="jamba-large", 
    tools=tools)
response.choices[0].message

AssistantMessage(role='assistant', content="I don't have the ability to answer general knowledge questions. My current capabilities are limited to performing mathematical operations such as addition and multiplication.", tool_calls=None)

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

In [10]:
messages = [
    ChatMessage(role="system", content="You are a helpful math assistant. Use the supplied tools to assist the user."),
    ChatMessage(role="user", content="Can you help me multiply 62.74 and 3.5?")
]

In [11]:
response = client.chat.completions.create(messages=messages, model="jamba-large", tools=tools)
response.choices[0].message

AssistantMessage(role='assistant', content=None, tool_calls=[ToolCall(id='chatcmpl-tool-c43a9211740c47a6930b2b997244ade6', function=ToolFunction(name='multiplication', arguments='{"a": 62.74, "b": 3.5}'), type='function')])

In [12]:
assistant_message = response.choices[0].message
tool_call = assistant_message.tool_calls[0]
tools_parameters_dict = json.loads(tool_call.function.arguments)

tool_result = multiplication(tools_parameters_dict["a"], tools_parameters_dict["b"])
tool_result

Multiplying 62.74 and 3.5...


219.59

In [13]:
tool_message = ToolMessage(role="tool", 
                           tool_call_id=tool_call.id, 
                           content=str(tool_result))

In [14]:
messages.append(assistant_message)
messages.append(tool_message)
response_final = client.chat.completions.create(
    messages=messages, 
    model="jamba-large")
response_final.choices[0].message.content

'The product of 62.74 and 3.5 is 219.59.'

In [15]:
tools_function = {"multiplication": multiplication, "addition": addition}
function_name = tool_call.function.name
tools_parameters = list(json.loads(tool_call.function.arguments).values())
tool_result = tools_function[function_name](*tools_parameters)
tool_result

Multiplying 62.74 and 3.5...


219.59

## SEC 10-Q analysis tool calling with Jamba

In [16]:
from utils import sec_10q

In [17]:
tool_definition = ToolDefinition(
    type="function",
    function=FunctionToolDefinition(
        name="sec_10q",
        description="Retrieve the full text of the latest 10-Q filing for a given company ticker",
        parameters=ToolParameters(
            type="object",
            properties={
                "ticker": {"type": "string", "description": "The ticker symbol of the company"}
            },
            required=["ticker"],
        ),
    ),
)

In [18]:
tools = [tool_definition]

In [19]:
messages = [
    ChatMessage(
        role="system",
        content="You are a helpful financial assistant. Use the supplied tools to assist the user with quarterly financial data.",
    ),
    ChatMessage(
        role="user", 
        content="Provide a summary of Nvidia's most recent quarterly 10-Q filing"),
]

In [20]:
response = client.chat.completions.create(
    messages=messages, 
    model="jamba-large", 
    tools=tools)
response

ChatCompletionResponse(id='chatcmpl-057dc9ad-1e58-3b29-8920-8e04a811d50c', choices=[ChatCompletionResponseChoice(index=0, message=AssistantMessage(role='assistant', content=None, tool_calls=[ToolCall(id='chatcmpl-tool-cdc0329827fb4f9eb910011f947f89f7', function=ToolFunction(name='sec_10q', arguments='{"ticker": "NVDA"}'), type='function')]), logprobs=None, finish_reason='tool_calls')], usage=UsageInfo(prompt_tokens=179, completion_tokens=31, total_tokens=210))

In [21]:
assistant_message = response.choices[0].message
tool_call = assistant_message.tool_calls[0]
tools_parameters_dict = json.loads(tool_call.function.arguments)

tool_result = sec_10q(tools_parameters_dict["ticker"])
tool_result

{'ticker': 'NVDA',
 'filing_type': '10-Q',
 'filing_url': 'https://www.sec.gov/Archives/edgar/data/1045810/000104581025000209/nvda-20250727.htm',
 'full_text': 'nvda-20250727 0001045810 1/25 2026 Q2 FALSE http://fasb.org/us-gaap/2025#AccruedLiabilitiesCurrent http://fasb.org/us-gaap/2025#AccruedLiabilitiesCurrent xbrli:shares iso4217:USD iso4217:USD xbrli:shares xbrli:pure nvda:investment nvda:segment 0001045810 2025-01-27 2025-07-27 0001045810 2025-08-22 0001045810 2025-04-28 2025-07-27 0001045810 2024-04-29 2024-07-28 0001045810 2024-01-29 2024-07-28 0001045810 2025-07-27 0001045810 2025-01-26 0001045810 us-gaap:CommonStockMember 2025-04-27 0001045810 us-gaap:AdditionalPaidInCapitalMember 2025-04-27 0001045810 us-gaap:AccumulatedOtherComprehensiveIncomeMember 2025-04-27 0001045810 us-gaap:RetainedEarningsMember 2025-04-27 0001045810 2025-04-27 0001045810 us-gaap:RetainedEarningsMember 2025-04-28 2025-07-27 0001045810 us-gaap:AccumulatedOtherComprehensiveIncomeMember 2025-04-28 2025-0

In [22]:
messages.append(assistant_message)

tool_message = ToolMessage(role="tool", 
                           tool_call_id=tool_call.id, 
                           content=json.dumps(tool_result))
messages.append(tool_message)

response_final = client.chat.completions.create(
    messages=messages, 
    model="jamba-large")
print(response_final.choices[0].message.content)

**Summary of NVIDIA's Q2 FY2026 10-Q Filing**

**Financial Performance**

- **Revenue**: $46.7 billion, up 56% year-over-year (YoY) and 6% sequentially.
- **Gross Profit**: $33.9 billion, with a gross margin of 72.4%, down slightly from 75.1% in Q2 FY2025.
- **Operating Expenses**: Increased 38% YoY to $5.4 billion, driven by higher R&D and SG&A costs.
- **Net Income**: $26.4 billion, up 59% YoY and 41% sequentially.
- **Earnings Per Share (EPS)**: Diluted EPS of $1.08, up 61% YoY.

**Segment Performance**

- **Compute & Networking**: Revenue of $41.3 billion, up 56% YoY, driven by demand for AI and data center solutions.
- **Graphics**: Revenue of $5.4 billion, up 51% YoY, with strong sales in gaming and professional visualization.

**Balance Sheet Highlights**

- **Cash and Cash Equivalents**: $11.6 billion, up from $8.6 billion in Q1 FY2026.
- **Total Assets**: $140.7 billion, including $45.2 billion in marketable securities.
- **Liabilities**: $40.6 billion, with $24.3 billion in c

In [23]:
messages = [
    ChatMessage(
        role="system",
        content="You are a helpful financial assistant. Use the supplied tools to assist the user with quarterly financial data.",
    ),
    ChatMessage(role="user", content="Provide a summary of Nvidia's most recent quarterly 10-Q filing"),
]
response = client.chat.completions.create(messages=messages, model="jamba-large", tools=tools)
assistant_message = response.choices[0].message
messages.append(assistant_message)

tool_result = None
tool_calls = assistant_message.tool_calls
if tool_calls:
    tool_call = tool_calls[0]
    if tool_call.function.name == "sec_10q":
        tools_parameters_dict = json.loads(tool_call.function.arguments)
        if "ticker" in tools_parameters_dict:
            tool_result = sec_10q(tools_parameters_dict["ticker"])
        else:
            print("Missing 'ticker' in function arguments")
    else:
        print(f"Unexpected tool call found - {tool_call.function.name}")
else:
    print("No tool calls found")

if tool_result is not None:
    tool_message = ToolMessage(role="tool", tool_call_id=tool_call.id, content=json.dumps(tool_result))
    messages.append(tool_message)
    response_final = client.chat.completions.create(messages=messages, model="jamba-large", tools=tools)

print("Final response:")
print(response_final.choices[0].message.content)

Final response:
NVIDIA's second-quarter 2026 financial results highlighted significant growth, with total revenue reaching $46.7 billion, a 56% increase compared to the same period last year. Data Center revenue led this growth, rising 56% year-over-year to $41.1 billion, driven by strong demand for accelerated computing platforms and AI solutions. Networking revenue also surged by 98% year-over-year to $7.3 billion, supported by the adoption of NVLink compute fabric and AI-focused Ethernet solutions.

Gaming revenue increased by 49% year-over-year, reflecting strong sales and improved supply of Blackwell products. Professional Visualization and Automotive revenue grew by 32% and 69% year-over-year, respectively, driven by AI workflows and self-driving platform adoption.

Geographically, revenue from customers outside the United States accounted for 50% of total revenue, with Singapore representing 22% of the total. Direct customers, including cloud service providers and system integra